### Once you have installed Conda, you will also want to run the following line in your Anaconda terminal:

#### conda install -c conda-forge pyglet

### Import dependencies

In [ ]:
!pip install stable-baselines3[extra]

In [ ]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

### Load Environment

In [ ]:
environment_name = "CartPole-v0"
env = gym.make(environment_name)

In [ ]:
episodes = 5
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render() # renders environment
        action = env.action_space.sample() # Discrete, picks an action, Move Left / Right
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close() # Close environment, can run outside of cell to also close environment

### Train RL Model

In [ ]:
# Make Directories To Save Logs (and later Models)
log_path = os.path.join('training', 'logs')

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
# Multilayer Perceptron Policy
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000)

### Save and Reload Model

In [ ]:
PPO_Path = os.path.join('training', 'saved_models', 'PPO_Model_Cartpole')

In [ ]:
model.save(PPO_Path)

In [ ]:
del model

In [ ]:
model = PPO.load(PPO_Path, env=env)

In [ ]:
model.learn(total_timesteps=1000)

### Evaluation

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

### Test Model

In [ ]:
episodes = 5
for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render() # renders environment
        action, _ = model.predict(obs) # predict returns two, but we only require action
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close() # Close environment, can run outside of cell to also close environment

### Viewing Logs with Tensorboard
#### Note: This should not be run within Jupyter, as it will freeze the notebook while it runs.
#### However, it can be run within the Notebook purely for demonstration purposes, so it will be included.

In [ ]:
training_log_path = os.path.join(log_path, 'PPO_2')

In [ ]:
#!tensorboard --logdir={training_log_path}
# if you are running this from jupyter, go to http://localhost:6006 to access

### Callback to Training Stage

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [ ]:
save_path = os.path.join('training', 'saved_models')

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000,
                             best_model_save_path= save_path,
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

### Changing Policies

In [ ]:
net_arch = [dict(pi=[128,128,128,128], vf=[128,128,128,128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch':net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

### Using an Alternative Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000)

In [ ]:
# To load DQN, do DQN.load instead of PPO.load